In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Define the dataset class
class InteractionDataset(Dataset):
    def __init__(self, user_ids, item_ids, interactions):
        self.user_ids = user_ids
        self.item_ids = item_ids
        self.interactions = interactions

    def __len__(self):
        return len(self.interactions)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.interactions[idx]

# Define the Neural Collaborative Filtering model
class NCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=8, hidden_dim=16):
        super(NCF, self).__init__()
        # User and item embeddings
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
        # Define hidden layers
        self.fc1 = nn.Linear(embedding_dim * 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.output = nn.Linear(hidden_dim // 2, 1)
        
        # Activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, user_id, item_id):
        # Get embeddings
        user_embed = self.user_embedding(user_id)
        item_embed = self.item_embedding(item_id)
        
        # Concatenate user and item embeddings
        x = torch.cat([user_embed, item_embed], dim=-1)
        
        # Forward pass through hidden layers
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        
        # Output layer with sigmoid
        return self.sigmoid(self.output(x))

# Example dataset
user_ids = np.array([0, 0, 1, 1, 2])
item_ids = np.array([0, 1, 0, 2, 3])
interactions = np.array([1, 0, 1, 1, 0])

# Hyperparameters
num_users = 3
num_items = 4
embedding_dim = 8
hidden_dim = 16
batch_size = 2
learning_rate = 0.01
epochs = 5

# Prepare the dataset and dataloader
dataset = InteractionDataset(user_ids, item_ids, interactions)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize model, loss function, and optimizer
model = NCF(num_users, num_items, embedding_dim, hidden_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for user_id, item_id, interaction in dataloader:
        # Convert to tensors
        user_id = torch.LongTensor(user_id)
        item_id = torch.LongTensor(item_id)
        interaction = torch.tensor(interaction, dtype=torch.float).view(-1, 1)  # Reshape interaction tensor
        
        # Forward pass
        predictions = model(user_id, item_id).squeeze()
        loss = criterion(predictions, interaction)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

# Example inference: Predict interaction for user 1 and item 2
user_id = torch.LongTensor([0])  # User ID 1 (0-based index)
item_id = torch.LongTensor([2])  # Item ID 3 (0-based index)
predicted_interaction = model(user_id, item_id).item()
print(f"Predicted interaction for User 1 and Item 3: {predicted_interaction:.4f}")

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_17112/2738543275.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  interaction = torch.tensor(interaction, dtype=torch.float).view(-1, 1)  # Reshape interaction tensor


ValueError: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2])) is deprecated. Please ensure they have the same size.